In [1]:
import sys; sys.path.append("..") 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.utility import *
from src.dataprocessing import *
from src import rnn

## load data

In [2]:
# Define variables
input_var   = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1"]
output_var  = ["TT-3061-3", "TT-3061-5", "LT-3061-2"]
process_var = input_var + output_var

In [3]:
min_len = 100
continuous_path = './data/3_continuous'

cts_list = []
i = 1
while exists(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv'):
    cts_df = loadfile(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv')
    cts_list.append(cts_df)
    i += 1

csv file is loaded from ./data/3_continuous/cts_100/dataset 100_1.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_2.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_3.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_4.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_5.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_6.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_7.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_8.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_9.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_10.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_11.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_12.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_13.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_14.csv
csv file is loa

In [4]:
target_list = cts_list

# history size and future size
history_list = [10, 20, 30, 40]
future_list = [10, 20, 30]
step = 1

# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# supervised attention factor
delta = 1
att_type = 'linear'
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers = 1
num_neurons = 50
dense_layers = 1
dense_neurons = 50
model_type = 'datt_seq2seq_gru'

for history_size in history_list:
    for future_size in future_list:
        print(f"history size: {history_size}")
        print(f"future size: {future_size}")
        history_series = []
        future_series = []

        # data to series
        for i in range(len(target_list)):
            history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                        step, start_idx=0, end_idx=None)
            if not i:
                history_series = history
                future_series = future
            else:
                history_series = np.concatenate([history_series, history], axis=0)
                future_series = np.concatenate([future_series, future], axis=0)
        
        factor = rnn.super_attention(delta, future_size, future_num, att_type)
        # Dual-attention Seq2Seq model
        DATT_seq2seq_GRU = rnn.RNN(history_series, history_var, future_series, future_var)
        # TT split
        DATT_seq2seq_GRU.train_test(test_size=test_size, test_num=test_num)
        # TV split
        valid_size = DATT_seq2seq_GRU.history_test.shape[0]/DATT_seq2seq_GRU.history_train.shape[0]
        DATT_seq2seq_GRU.train_valid(valid_size=valid_size)
        # scaling
        DATT_seq2seq_GRU.scaling()
        # modeling
        DATT_seq2seq_GRU.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
        # training
        model_num = 'test'
        model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{model_num}"
        if not exists("./model", model_name, 'model'):
            DATT_seq2seq_GRU.train()
            DATT_seq2seq_GRU.save_model(f"./model/{model_name}")
        else:
            DATT_seq2seq_GRU.model = loadfile("./model", model_name, 'model')
        # test
        test_result = DATT_seq2seq_GRU.test()
        print("end training")
        
print('end opitmization')

history size: 10
future size: 10
Epoch 1/10000
581/581 - 10s - loss: 0.2449 - val_loss: 0.1585 - 10s/epoch - 17ms/step
Epoch 2/10000
581/581 - 5s - loss: 0.1506 - val_loss: 0.1376 - 5s/epoch - 8ms/step
Epoch 3/10000
581/581 - 5s - loss: 0.1410 - val_loss: 0.1411 - 5s/epoch - 8ms/step
Epoch 4/10000
581/581 - 5s - loss: 0.1324 - val_loss: 0.1270 - 5s/epoch - 8ms/step
Epoch 5/10000
581/581 - 5s - loss: 0.1238 - val_loss: 0.1136 - 5s/epoch - 8ms/step
Epoch 6/10000
581/581 - 5s - loss: 0.1181 - val_loss: 0.1062 - 5s/epoch - 8ms/step
Epoch 7/10000
581/581 - 5s - loss: 0.1126 - val_loss: 0.1048 - 5s/epoch - 8ms/step
Epoch 8/10000
581/581 - 5s - loss: 0.1078 - val_loss: 0.0990 - 5s/epoch - 9ms/step
Epoch 9/10000
581/581 - 5s - loss: 0.1030 - val_loss: 0.0992 - 5s/epoch - 9ms/step
Epoch 10/10000
581/581 - 5s - loss: 0.1003 - val_loss: 0.0938 - 5s/epoch - 9ms/step
Epoch 11/10000
581/581 - 5s - loss: 0.0970 - val_loss: 0.0921 - 5s/epoch - 9ms/step
Epoch 12/10000
581/581 - 5s - loss: 0.0950 - val_

INFO:tensorflow:Assets written to: ./model/10_10_1_50_1_50_datt_seq2seq_gru_test\assets


INFO:tensorflow:Assets written to: ./model/10_10_1_50_1_50_datt_seq2seq_gru_test\assets


,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.94855,0.954594,0.967859,0.957001,3.682886,3.726303,2.795242,3.401477
1,0.878513,0.904571,0.918817,0.900634,5.623009,5.402695,4.441325,5.155677
2,0.784107,0.855841,0.854321,0.831423,7.393895,6.640935,5.948003,6.660944
3,0.685083,0.816574,0.8042,0.768619,8.811611,7.492033,6.893932,7.732525
4,0.587953,0.784423,0.779874,0.717416,9.980797,8.123277,7.307314,8.470463
5,0.4959,0.761174,0.774487,0.677187,10.933266,8.551523,7.394329,8.959706
6,0.408534,0.744815,0.776359,0.643236,11.730636,8.840315,7.362248,9.311066
7,0.328695,0.730533,0.777887,0.612371,12.467636,9.084667,7.33633,9.629545
8,0.259053,0.717671,0.772577,0.583101,13.113544,9.299384,7.422933,9.945287


history size: 10
future size: 20
Epoch 1/10000
577/577 - 8s - loss: 0.3112 - val_loss: 0.2645 - 8s/epoch - 14ms/step
Epoch 2/10000
577/577 - 5s - loss: 0.2273 - val_loss: 0.2372 - 5s/epoch - 8ms/step
Epoch 3/10000
577/577 - 4s - loss: 0.2123 - val_loss: 0.2300 - 4s/epoch - 8ms/step
Epoch 4/10000
577/577 - 4s - loss: 0.2036 - val_loss: 0.2134 - 4s/epoch - 7ms/step
Epoch 5/10000
577/577 - 4s - loss: 0.1929 - val_loss: 0.2111 - 4s/epoch - 7ms/step
Epoch 6/10000
577/577 - 4s - loss: 0.1854 - val_loss: 0.1975 - 4s/epoch - 7ms/step
Epoch 7/10000
577/577 - 4s - loss: 0.1775 - val_loss: 0.1913 - 4s/epoch - 7ms/step
Epoch 8/10000
577/577 - 4s - loss: 0.1723 - val_loss: 0.1910 - 4s/epoch - 7ms/step
Epoch 9/10000
577/577 - 4s - loss: 0.1655 - val_loss: 0.1822 - 4s/epoch - 7ms/step
Epoch 10/10000
577/577 - 4s - loss: 0.1613 - val_loss: 0.1812 - 4s/epoch - 7ms/step
Epoch 11/10000
577/577 - 4s - loss: 0.1555 - val_loss: 0.1699 - 4s/epoch - 7ms/step
Epoch 12/10000
577/577 - 4s - loss: 0.1500 - val_lo

...

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.9091,0.913993,0.951672,0.924921,5.948916,5.156673,3.376807,4.827465
1,0.815344,0.865295,0.921333,0.867324,8.326526,6.455542,4.309538,6.363869
2,0.701051,0.824692,0.885894,0.803879,10.133035,7.367321,5.191751,7.564036
3,0.585628,0.793889,0.859187,0.746235,11.531438,7.991198,5.768685,8.43044
4,0.47862,0.767537,0.843643,0.6966,12.727887,8.49047,6.080141,9.099499
5,0.381571,0.744129,0.834163,0.653287,13.815178,8.911728,6.262828,9.663245
6,0.294258,0.722368,0.825022,0.613883,14.46882,9.28506,6.434417,10.062766
7,0.215872,0.701731,0.814784,0.577462,14.959615,9.62612,6.621284,10.40234
8,0.148178,0.682803,0.802885,0.544622,15.361607,9.928587,6.831881,10.707358


history size: 40
future size: 30
Epoch 1/10000
559/559 - 8s - loss: 0.3749 - val_loss: 0.3162 - 8s/epoch - 14ms/step
Epoch 2/10000
559/559 - 4s - loss: 0.2851 - val_loss: 0.2826 - 4s/epoch - 8ms/step
Epoch 3/10000
559/559 - 4s - loss: 0.2635 - val_loss: 0.2686 - 4s/epoch - 8ms/step
Epoch 4/10000
559/559 - 4s - loss: 0.2470 - val_loss: 0.2540 - 4s/epoch - 8ms/step
Epoch 5/10000
559/559 - 4s - loss: 0.2293 - val_loss: 0.2348 - 4s/epoch - 8ms/step
Epoch 6/10000
559/559 - 4s - loss: 0.2153 - val_loss: 0.2112 - 4s/epoch - 8ms/step
Epoch 7/10000
559/559 - 4s - loss: 0.2021 - val_loss: 0.2027 - 4s/epoch - 8ms/step
Epoch 8/10000
559/559 - 4s - loss: 0.1894 - val_loss: 0.1956 - 4s/epoch - 8ms/step
Epoch 9/10000
559/559 - 4s - loss: 0.1793 - val_loss: 0.1839 - 4s/epoch - 8ms/step
Epoch 10/10000
559/559 - 4s - loss: 0.1728 - val_loss: 0.1794 - 4s/epoch - 8ms/step
Epoch 11/10000
559/559 - 4s - loss: 0.1657 - val_loss: 0.1761 - 4s/epoch - 8ms/step
Epoch 12/10000
559/559 - 4s - loss: 0.1615 - val_lo

INFO:tensorflow:Assets written to: ./model/40_30_1_50_1_50_datt_seq2seq_gru_test\assets


INFO:tensorflow:Assets written to: ./model/40_30_1_50_1_50_datt_seq2seq_gru_test\assets


,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.866964,0.864255,0.911928,0.881049,7.195307,6.478311,4.545779,6.073132
1,0.76892,0.823652,0.872457,0.821677,9.485051,7.384375,5.473092,7.447506
2,0.662352,0.787553,0.841929,0.763945,11.467871,8.105745,6.095866,8.556494
3,0.567103,0.755747,0.813093,0.711981,12.987207,8.692672,6.63119,9.437023
4,0.482726,0.73124,0.787828,0.667264,14.197117,9.120146,7.067593,10.128285
5,0.409226,0.711687,0.764263,0.628392,15.17147,9.44845,7.451604,10.690508
6,0.347392,0.695034,0.735714,0.592713,15.944273,9.719812,7.891768,11.185284
7,0.294446,0.678729,0.699114,0.557429,16.577138,9.978492,8.422652,11.659427
8,0.24924,0.658992,0.655026,0.521086,17.098908,10.283169,9.020686,12.134254
